In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import os
import math

In [ ]:
learning_rate = 0.0001
batch_size = 16

IMG_WIDTH = 208
IMG_HEIGHT = 208

In [ ]:
def get_label(file_path):
    # return whether the file_path is a cat or dog
    # (./train/dog.11971.jpg) return false
    # Look into string_split and tf.equal -> (compare tensor against python string)
    # Example is tf.string_split([file_path], 't')
    pass
def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    # return img that is all of the same size (right now they are not)
    # make sure all images are of IMG_WIDTH and IMG_HEIGHT
    # look into tf.image.resize_image_with_crop_or_pad
    pass
def process_path(file_path):
    # Main one that given a file_path (./train/dog.11971.jpg) should 
    # return
    # a tuple of image, label (matrix for the dog image, False for dog)
    # look into tf.io.read_file
    pass

In [ ]:
train_dir = "./train/"
# Creates a dataset (this is a graph operation not part of session)
# Dataset is full of strings right now corresponding to the file paths of the images
dataset = tf.data.Dataset.list_files(train_dir + "*.*.jpg")
# apply process path to each of the file paths that we got
dataset = dataset.map(process_path)
dataset = dataset.shuffle(buffer_size=500)
# Tru increasing batch size if your ram can handle it
dataset = dataset.batch(4)

iterator = dataset.make_initializable_iterator()
batch = iterator.get_next()

In [ ]:
with tf.Session() as sess:
    # Initialize our dataset
    sess.run(iterator.initializer)
    # Grab one batch
    print(sess.run(batch))

In [ ]:
def inference(images):
    # assume images is a placeholder
    # conv net with 
    # layer 1: 32 (5-by-5) kernels, relu
    # max pooling in between
    # layer 2: 64 (5-by-5) kernels, relu
    # max pooling in between
    # flatten the kernels so we can feed it into a dense layer
    # layer 3: dense layer with 1024 neuraons
    # layer 4: output dense layer with 2 neurons for the two classes
    return output_layer

In [ ]:
def losses(logits, labels):
    # sparse_softmax_cross_entropy_with_logits
    # make sure to reduce_mean
    return loss
def training(loss, learning_rate):
    # Use adam optimizer
    return train_op
def evaluation(logits, labels):
    # return how good the model is as a percentage (look into tf.nn.in_top_k)
    # make sure to reduce_mean to get a single value
    return accuracy


In [ ]:
x = tf.placeholder(tf.float32, shape=[None, IMG_WIDTH, IMG_HEIGHT, 3])
y = tf.placeholder(tf.int32, shape=[None])

model = inference(x)
loss = losses(model, y)  
acc = evaluation(model, y)
train_op = training(loss, learning_rate)

In [ ]:
epochs = 10
# This will take a while so if you want I can run this on a GPU for you. 
with tf.Session(config=config) as sess:
    # initialize all variables (intialize values in the neural network)
    sess.run(tf.global_variables_initializer())
    # Create a saver to save the model
    saver = tf.train.Saver()
    for epoch in range(epochs):
        while True:
            try:
                # grab images and labels with sess.run(batch)
                # Then feed it into network to get loss and accuracy
                # Print your loss and accuracy every 20 steps of training
            except tf.errors.OutOfRangeError:
                break
        save_path = saver.save(sess, "save/model" + str(epoch) + ".ckpt")